# DeerLab Workshop 2023
## EFEPR Summer School
## Hugo Karas

## What is DeerLab?

DeerLab is an advanced python based software for analysing DEER/PELDOR data using Tikihnov regularisation

## Getting Started

First we will look at processing a simple 4-pulse DEER trace

In [ ]:
import deerlab as dl
# We all need some other packages too
import numpy as np # Numerical calculations and array handling
import matplotlib.pyplot as plt # Plotting

In [ ]:
t,V = dl.deerload('../data/example_4pdeer_4.DTA')

# Plot the data
plt.figure(figsize=(8,3))
plt.plot(t,V.real,'.',label='data (real)')
plt.plot(t,V.imag,'.',label='data (imag)')
plt.xlabel('t (µs)')
plt.ylabel('V (arb.u.)')
plt.legend()
plt.show()

## Pre-processing Data

In [ ]:
# Phase correction
Vexp = dl.correctphase(V)
# Normalize the data (only for plotting purposes)
Vexp = Vexp/np.max(Vexp)
plt.figure(figsize=(8,3))
plt.plot(t,Vexp,'.',label='data corr.')
plt.xlabel('t (µs)')
plt.ylabel('V (arb.u.)')
plt.legend()
plt.show()

### Acounting for the start time

Since DeerLab has been designed for multi-pathway data, the definition of the zero point in time is critically important

<img style="width:500px; alignment:center" src="https://raw.githubusercontent.com/JeschkeLab/DeerLabWorkshop2022/main/images/4pdeer_sequence_starttime.svg">

In Deerlab the *zerotime* is defined as the time of $\pi$ pulse before the moving pump pulse


In [ ]:
tau1 = 0.5 # µs
tau2 = 5.0 # µs
deadtime = 0.3 # µs

t = t -t[0]
t = t + deadtime

plt.figure(figsize=(8,3))
plt.plot(t,Vexp,'.',label='data corr.')

plt.vlines(0,min(Vexp),max(Vexp),'k',linestyle='dashed')
plt.vlines(tau1+tau2,min(Vexp),max(Vexp),'k',linestyle='dashed')
plt.vlines(tau1,min(Vexp),max(Vexp),'r',linestyle='dashed')


plt.xlabel('t (µs)')
plt.ylabel('V (arb.u.)')
plt.legend()
plt.show()

## Fitting

Before we can fit out data we must first build a model.

We can build a simple dipolar model

In [ ]:
# Define the distance range
r = np.linspace(2,5,50) # range from 2.2-5.2nm with 0.02nm resolution

# Construct the dipolar model
Vmodel = dl.dipolarmodel(t,r) 

# Display information about the dipolar model
print(Vmodel)

DeerLab includes a wide range of built in models which can then handle the different pathways.

- 4pDEER
- fwd5pDEER
- rev5pDEER
- RIDME

and many more...

In [ ]:
# Construct model of our 4-pulse DEER experiment
my4pdeer = dl.ex_4pdeer(tau1,tau2,pathways=[1,2,3])

# Construct the dipolar signal model including experimental information
Vmodel = dl.dipolarmodel(t,r, experiment=my4pdeer) 

# Display information about the model
print(Vmodel)

We can now use this model to fit our data in one go

In [ ]:
results = dl.fit(Vmodel,Vexp,verbose=2,  ftol=1e-3,)

In [ ]:
print(results)

In [ ]:
results.plot(axis=t,xlabel='t [μs]')

# Show the plot
plt.show() 

In [ ]:
# Get the fitted distance distribution
Pfit = results.P
# Get the uncertainty quantification of the distance distribution
Puq = results.PUncert
# Get its 95% confidence intervals
Pci95 = Puq.ci(95)

# Get the lower/upper boundaries of the confidence intervals
Pci95_lower = Pci95[:,0]
Pci95_upper = Pci95[:,1]

# Plot the fitted distance distribution
plt.plot(r,Pfit)
# Plot the confidence bands as a filled area
plt.fill_between(r,Pci95_lower,Pci95_upper,alpha=0.4)
plt.xlabel('r [nm]')
plt.ylabel('P(r) [nm$^{-1}$]')
plt.show()

In [ ]:
mod_fit = results.lam1 + results.lam2 + results.lam3
mod_ci95 = results.lam1Uncert.ci(95) + results.lam2Uncert.ci(95) + results.lam3Uncert.ci(95)

# Spin concentration
conc_fit = results.conc
conc_ci95 = results.concUncert.ci(95)


print(f'Modulation depth: {mod_fit:.3f} ({mod_ci95[0]:.3f},{mod_ci95[1]:.3f})')
print(f'Spin concentration: {conc_fit:.4g}uM ({conc_ci95[0]:.4g},{conc_ci95[1]:.4g})')

In [ ]:
stats = dl.diststats(r,Pfit,Puq,verbose=True)

## Advanced Techniques

- Custom Dipolar Modelling
- Compactness Criterion
- Global Fitting

## Custom Dipolar Modeling

## Compactness Criterion


It is often the case that there is a non-unique solution for the speration of the background. This is especially tue for noisy data and dataset where the trace is not long enough.

<img src="https://raw.githubusercontent.com/JeschkeLab/DeerLabWorkshop2022/2746d801bdf97b7afd75d4f666d8fd414af490d8/images/example_identiability.svg" style="width:700px">

DeerLab has a compactness penalty introduced in the regularization. This has the effect of adding an aditional penalty to the objective function than penalizes non-compact data. 

## Global Fitting